In [1]:
%pip install cryptography

Note: you may need to restart the kernel to use updated packages.


In [2]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.serialization import Encoding, ParameterFormat, PublicFormat
from cryptography.hazmat.primitives.serialization import load_pem_parameters, load_pem_public_key
from cryptography.hazmat.primitives.kdf.hkdf import HKDF

In [3]:
with open ("shared_file", "rb") as file:
    parameters = load_pem_parameters(file.read()) 

In [4]:
private_key_D = parameters.generate_private_key() # Blue
public_mixed_with_D = private_key_D.public_key()  # light-blue-green

with open("shared_file", "wb") as file:
    file.write(public_mixed_with_D.public_bytes(encoding=Encoding.PEM, format=PublicFormat.SubjectPublicKeyInfo))

In [5]:
with open ("shared_file", "rb") as file:
    public_mixed_with_C = load_pem_public_key(file.read()) 

In [6]:
shared_key_C_then_D = private_key_D.exchange(public_mixed_with_C) # brown

derived_key_C_then_D = HKDF(
    algorithm = hashes.SHA256(),
    length=32,
    salt=None,
    info=b'handshake data',
    backend=default_backend()
).derive(shared_key_C_then_D)

print(derived_key_C_then_D)

b'\xdfm\xc0\x9d\x03\xbc\xa7\xb4\xaeU\xf2\xf0\x1eQ\x8d}ty\xc4\xd9\xc5\xaf\x1a\xd9\x10JW\t=:\x9e\x1d'


### Decryption

In [12]:
from Crypto.Cipher import AES
from Crypto.Util import Counter
from io import StringIO
import pandas as pd

In [13]:
with open ("shared_file", "rb") as file:
    columns_desc_enc = file.read()

def decrypt_csv_ctr(ct, key, counter_iv, counter_size=128):
    counter = Counter.new(counter_size, initial_value=counter_iv)    
    aes = AES.new(key=key, mode=AES.MODE_CTR, counter=counter)
    return aes.decrypt(ct)

pd.read_csv(StringIO(decrypt_csv_ctr(columns_desc_enc, derived_key_C_then_D, 10).decode('utf-8')))

,New name,Description
0,address,Client's address
1,age,Client's age in days at the time of application
2,annual_income,Income of the client
3,appendix_a,Did client provide document 2
4,appendix_b,Did client provide document 3
...,...,...
62,provided_workphone,"Did client provide work phone (1=YES, 0=NO)"
63,region_rating,Our rating of the region where client lives (1...
64,score_ext_1,Normalized score from external data source
65,score_ext_2,Normalized score from external data source


In [18]:
with open ("shared_file", "rb") as file:
    infringement_dataset_enc = file.read()

pd.read_csv(StringIO(decrypt_csv_ctr(infringement_dataset_enc, derived_key_C_then_D, 10).decode('utf-8')))

,loan_id,infringed,contract_type,gender,has_own_car,has_own_realty,num_children,annual_income,credit_amount,credit_annuity,...,SK_ID_CURR,avg_days_decision,past_avg_amount_annuity,past_avg_amt_application,past_avg_amt_credit,past_loans_approved,past_loans_refused,past_loans_canceled,past_loans_unused,past_loans_total
0,100002,1,Cash loans,M,N,Y,0,202500.0,406597.5,24700.5,...,100002.0,606.000000,9251.775000,179055.000,179055.00,1.0,0.0,0.0,0.0,1.0
1,100003,0,Cash loans,F,N,N,0,270000.0,1293502.5,35698.5,...,100003.0,1305.000000,56553.990000,435436.500,484191.00,3.0,0.0,0.0,0.0,3.0
2,100004,0,Revolving loans,M,Y,Y,0,67500.0,135000.0,6750.0,...,100004.0,815.000000,5357.250000,24282.000,20106.00,1.0,0.0,0.0,0.0,1.0
3,100006,0,Cash loans,F,N,Y,0,135000.0,312682.5,29686.5,...,100006.0,272.444444,23651.175000,272203.260,291695.50,5.0,3.0,1.0,0.0,9.0
4,100007,0,Cash loans,M,N,Y,0,121500.0,513000.0,21865.5,...,100007.0,1222.833333,12278.805000,150530.250,166638.75,6.0,0.0,0.0,0.0,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
307506,456251,0,Cash loans,M,N,N,0,157500.0,254700.0,27558.0,...,456251.0,273.000000,6605.910000,40455.000,40455.00,1.0,0.0,0.0,0.0,1.0
307507,456252,0,Cash loans,F,N,Y,0,72000.0,269550.0,12001.5,...,456252.0,2497.000000,10074.465000,57595.500,56821.50,1.0,0.0,0.0,0.0,1.0
307508,456253,0,Cash loans,F,N,Y,0,153000.0,677664.0,29979.0,...,456253.0,2380.000000,4770.405000,24162.750,20625.75,2.0,0.0,0.0,0.0,2.0
307509,456254,1,Cash loans,F,N,Y,0,171000.0,370107.0,20205.0,...,456254.0,299.500000,10681.132500,121317.750,134439.75,2.0,0.0,0.0,0.0,2.0
